Tujuan dari projek ini adalah untuk memperdalam pemahaman tentang statistik dengan Python,
data didapat dari https://www.kaggle.com/datasets/new-york-city/nyc-property-sales, data sudah didownload dan berada di D:\CODING\Python\Coursera\Data Structures\Hacktiv8\nyc-rolling-sales.

Referensi Kolom:
* BOROUGH: Manhattan(1), Bronx(2), Brooklyn(3), Queens(4), Staten Island(5)
* BLOCK;LOT: Kombinasi angka borough,blok dan lot biasa dijadikan alamat unik untuk bangunan di New York,biasa disebut BBL
* BUILDING CLASS AT PRESENT dan BUILDING CLASS AT TIME OF SALE: Tipe bangunan saat dijual, bisa apartemen,rumah,komersil dan lainnya

Akan dianalisa:
1. Adakah perbedaan harga pada bangunan yang dibangun pada tahun 1900 - 2000 dan setelah tahun 2000?
2. Berapa proporsi dan confidence interval dari bangunan yang dibangun setelah tahun 2000?
3. Apakah benar lebih banyak bangunan yang dibangun pada tahun lebih dari 2000 daripada bangunan yang dibangun pada tahun 1900 - 2000?


In [1]:
#import package
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')

import scipy.stats

import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ztest

In [2]:
#import data
pd_sales = pd.read_csv("nyc-rolling-sales.csv")
pd_sales.head(10)
                       

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
5,9,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,405,16,,C4,516 EAST 12TH STREET,...,20,0,20,2581,9730,1900,2,C4,-,2017-07-20 00:00:00
6,10,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,,C4,210 AVENUE B,...,8,0,8,1750,4226,1920,2,C4,3192840,2016-09-23 00:00:00
7,11,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,407,18,,C7,520 EAST 14TH STREET,...,44,2,46,5163,21007,1900,2,C7,-,2017-07-20 00:00:00
8,12,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,379,34,,D5,141 AVENUE D,...,15,0,15,1534,9198,1920,2,D5,-,2017-06-20 00:00:00
9,13,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,,D9,629 EAST 5TH STREET,...,24,0,24,4489,18523,1920,2,D9,16232000,2016-11-07 00:00:00


In [3]:
pd_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [4]:
pd_sales['SALE PRICE'].unique()

array(['6625000', ' -  ', '3936272', ..., '408092', '11693337', '69300'],
      dtype=object)

karena ada selain angka, rubah menjadi 0

In [5]:
pd_sales['SALE PRICE'].replace(' -  ','0',inplace=True)

In [6]:
pd_sales['SALE PRICE'].unique()

array(['6625000', '0', '3936272', ..., '408092', '11693337', '69300'],
      dtype=object)

In [7]:
#rubah dari type object menjadi integer
pd_sales = pd_sales.astype({'SALE PRICE':'int64'})
pd_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [8]:
pd_sales['BUILDING CLASS CATEGORY'].unique()

array(['07 RENTALS - WALKUP APARTMENTS             ',
       '08 RENTALS - ELEVATOR APARTMENTS           ',
       '09 COOPS - WALKUP APARTMENTS               ',
       '10 COOPS - ELEVATOR APARTMENTS             ',
       '11A CONDO-RENTALS                           ',
       '12 CONDOS - WALKUP APARTMENTS              ',
       '13 CONDOS - ELEVATOR APARTMENTS            ',
       '14 RENTALS - 4-10 UNIT                     ',
       '15 CONDOS - 2-10 UNIT RESIDENTIAL          ',
       '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT ',
       '17 CONDO COOPS                             ',
       '22 STORE BUILDINGS                         ',
       '37 RELIGIOUS FACILITIES                    ',
       '42 CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC  ',
       '46 CONDO STORE BUILDINGS                   ',
       '47 CONDO NON-BUSINESS STORAGE              ',
       '01 ONE FAMILY DWELLINGS                    ',
       '02 TWO FAMILY DWELLINGS                    ',
       '03 THREE FAMILY DWE

In [9]:
pd_sales['YEAR BUILT'].unique()

array([1900, 1913, 1920, 1910, 2009, 1925, 1902, 1928, 1930, 1935, 1937,
       1915, 1950, 1929, 1901, 1940, 2005,    0, 1989, 2014, 2008, 1965,
       2013, 2003, 2006, 2007, 1951, 1899, 1850, 1905, 1864, 1917, 1911,
       1983, 1926, 1963, 1960, 1889, 1898, 1939, 1938, 1927, 1909, 1958,
       1904, 1907, 1987, 1931, 1984, 1948, 2004, 1918, 1875, 2012, 1973,
       2011, 1922, 2001, 1932, 1980, 1908, 1953, 1906, 2015, 1946, 1921,
       2010, 1954, 1111, 1924, 1990, 1890, 1991, 1988, 1895, 2016, 1957,
       1986, 1966, 1998, 1870, 1923, 1969, 2017, 1968, 1934, 1956, 1982,
       1914, 1903, 1967, 1840, 1912, 1964, 1955, 1961, 1851, 2000, 1959,
       1962, 1945, 1972, 1976, 1916, 1880, 1970, 1846, 1941, 1952, 1896,
       1985, 1981, 1888, 1947, 1975, 1974, 2002, 1994, 1892, 1894, 1891,
       1996, 1997, 1949, 1999, 1800, 1979, 1971, 1977, 1942, 1978, 1826,
       1881, 1919, 1883, 1936, 1993, 1995, 1933, 1992, 1943, 1944, 1847,
       1829, 1844, 1835, 1852, 1856, 1854, 1832, 18

### Adakah perbedaan harga pada bangunan yang dibangun pada tahun 1900 - 2000 dan setelah tahun 2000?


In [10]:
#yang dibutuhkan adalah harga bangunan pada tahun sebelum 2000 dan setelah 2000,  ambil harga tahun itu saja
#ambil tahun 1900 - 2000
kon_year1 = (pd_sales['YEAR BUILT'] <= 2000 )
kon_year1

0         True
1         True
2         True
3         True
4         True
         ...  
84543     True
84544     True
84545     True
84546    False
84547    False
Name: YEAR BUILT, Length: 84548, dtype: bool

In [11]:
kon_year2 = (pd_sales['YEAR BUILT'] >= 1900)
kon_year2

0        True
1        True
2        True
3        True
4        True
         ... 
84543    True
84544    True
84545    True
84546    True
84547    True
Name: YEAR BUILT, Length: 84548, dtype: bool

In [12]:
pd_sales2 = pd_sales[(kon_year1 & kon_year2)]
pd_sales2.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,0,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,0,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [13]:
#cek apakah tahunnya benar 
pd_sales2['YEAR BUILT'].unique()

array([1900, 1913, 1920, 1910, 1925, 1902, 1928, 1930, 1935, 1937, 1915,
       1950, 1929, 1901, 1940, 1989, 1965, 1951, 1905, 1917, 1911, 1983,
       1926, 1963, 1960, 1939, 1938, 1927, 1909, 1958, 1904, 1907, 1987,
       1931, 1984, 1948, 1918, 1973, 1922, 1932, 1980, 1908, 1953, 1906,
       1946, 1921, 1954, 1924, 1990, 1991, 1988, 1957, 1986, 1966, 1998,
       1923, 1969, 1968, 1934, 1956, 1982, 1914, 1903, 1967, 1912, 1964,
       1955, 1961, 2000, 1959, 1962, 1945, 1972, 1976, 1916, 1970, 1941,
       1952, 1985, 1981, 1947, 1975, 1974, 1994, 1996, 1997, 1949, 1999,
       1979, 1971, 1977, 1942, 1978, 1919, 1936, 1993, 1995, 1933, 1992,
       1943, 1944], dtype=int64)

In [14]:
#untuk tahun setelah 2000
kon_year3 = (pd_sales['YEAR BUILT'] > 2000)
kon_year3

0        False
1        False
2        False
3        False
4        False
         ...  
84543    False
84544    False
84545    False
84546     True
84547     True
Name: YEAR BUILT, Length: 84548, dtype: bool

In [15]:
pd_sales3 = pd_sales[kon_year3]
pd_sales3.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
11,15,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2B,400,21,,D1,530-532 EAST 5TH STREET,...,10,0,10,3717,12350,2009,2,D1,10350000,2016-10-17 00:00:00
73,77,1,ALPHABET CITY,11A CONDO-RENTALS,2,385,1102,,RR,254 EAST 2ND STREET,...,47,0,47,-,36472,2005,2,RR,28000000,2016-10-13 00:00:00
88,92,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,378,1001,,R4,399 EAST 8TH STREET,...,1,0,1,-,-,2014,2,R4,0,2017-05-25 00:00:00
89,93,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,378,1002,,R4,399 EAST 8TH STREET,...,1,0,1,-,-,2014,2,R4,0,2017-05-25 00:00:00
90,94,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,378,1003,,R4,399 EAST 8TH STREET,...,1,0,1,-,-,2014,2,R4,0,2017-07-14 00:00:00


In [16]:
#cek apakah tahunnya benar
pd_sales3['YEAR BUILT'].unique()

array([2009, 2005, 2014, 2008, 2013, 2003, 2006, 2007, 2004, 2012, 2011,
       2001, 2015, 2010, 2016, 2017, 2002], dtype=int64)

In [17]:
pd_sales3['SALE PRICE'].unique()

array([10350000, 28000000,        0, ...,   772500, 11693337,    69300],
      dtype=int64)

In [18]:
#cari mean median modus dari masing masing rentang tahun
#untuk tahun 1900 - 2000
pd_sales2['SALE PRICE'].mean()

978128.0359717868

In [19]:
pd_sales2['SALE PRICE'].median()


389547.0

In [20]:
pd_sales2['SALE PRICE'].mode()

0    0
Name: SALE PRICE, dtype: int64

In [21]:
#untuk tahun setelah 2000
pd_sales3['SALE PRICE'].mean()

1450035.288530466

In [22]:
pd_sales3['SALE PRICE'].median()

623065.0

In [23]:
pd_sales3['SALE PRICE'].mode()

0    0
Name: SALE PRICE, dtype: int64

dari mean, median, dan mode didapat
PYTN_2_098_Athiyyah Nisrina Husna

Akan dicari Range, Variance, dan  Standard Deviation dari keduanya

In [24]:
#Standar Deviasi
pd_sales2['SALE PRICE'].std()

11516650.115237629

In [25]:
pd_sales3['SALE PRICE'].std()

5286072.141200107

In [26]:
#variansi
pd_sales2['SALE PRICE'].var()

132633229876802.88

In [27]:
pd_sales3['SALE PRICE'].var()

27942558681971.875

Bangunan yang dibangun pada tahun 1900 - 2000 dijual pada kisaran 11516650.115237629 sedangkan bangunan yang dibangun pada tahun setelah 2000 dijual pada kisaran 5286072.141200107

In [28]:
#Range
print("min", pd_sales2['SALE PRICE'].min())
print("max", pd_sales2['SALE PRICE'].max())
print("range", pd_sales2['SALE PRICE'].max() - pd_sales2['SALE PRICE'].min())

min 0
max 2210000000
range 2210000000


In [29]:
print("min", pd_sales3['SALE PRICE'].min())
print("max", pd_sales3['SALE PRICE'].max())
print("range", pd_sales3['SALE PRICE'].max() - pd_sales3['SALE PRICE'].min())

min 0
max 203345000
range 203345000


Karena ada bangunan yang dijual dengan harga 0 (transaksi balik nama) maka range nya menjadi harga paling tinggi

### Berapa proporsi dan confidence interval dari bangunan yang dibangun setelah tahun 2000?

In [30]:
#sudah ada dataframe nya yaitu pd_sales3
#cari jumlah bangunannya
pd_sales3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11718 entries, 11 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      11718 non-null  int64 
 1   BOROUGH                         11718 non-null  int64 
 2   NEIGHBORHOOD                    11718 non-null  object
 3   BUILDING CLASS CATEGORY         11718 non-null  object
 4   TAX CLASS AT PRESENT            11718 non-null  object
 5   BLOCK                           11718 non-null  int64 
 6   LOT                             11718 non-null  int64 
 7   EASE-MENT                       11718 non-null  object
 8   BUILDING CLASS AT PRESENT       11718 non-null  object
 9   ADDRESS                         11718 non-null  object
 10  APARTMENT NUMBER                11718 non-null  object
 11  ZIP CODE                        11718 non-null  int64 
 12  RESIDENTIAL UNITS               11718 non-null  in

In [31]:
n_bangunanall = 84548 #dari pd_sales
n_bangunan1 = 11718 #dari pd_sales3

In [32]:
#cari proporsi
p_bangunan1 = n_bangunan1 / n_bangunanall
p_bangunan1

0.1385958272224062

dari dataset kita, 13% bangunan dibangun pada tahun setelah 2000

In [33]:
#confidence interval
# z adalah angka konstan untuk menghitung margin error, dimana tingkat keyakinan 95% artinya z=1.96
z = 1.96

# margin error dihitung dengan z * akar(p * (1-p) / jumlah_data)
margin_error = z * np.sqrt(p_bangunan1* (1-p_bangunan1) / n_bangunanall)

# confidence interval adalah probabilitas +- margin error
print(p_bangunan1 - margin_error, p_bangunan1 + margin_error)

0.13626675495038024 0.14092489949443218


13% - 14% bangunan dibangun setelah tahun 2000

### Apakah benar lebih banyak bangunan yang dibangun pada tahun lebih dari 2000 daripada bangunan yang dibangun pada tahun 1900 - 2000?

H0 = p_bangunan1 < p_bangunan2

H1 = p_bangunan1 > p_bangunan2

In [34]:
#jumlah bangunan yang dibangun pada tahun 1900 - 2000
pd_sales2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63800 entries, 0 to 84545
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      63800 non-null  int64 
 1   BOROUGH                         63800 non-null  int64 
 2   NEIGHBORHOOD                    63800 non-null  object
 3   BUILDING CLASS CATEGORY         63800 non-null  object
 4   TAX CLASS AT PRESENT            63800 non-null  object
 5   BLOCK                           63800 non-null  int64 
 6   LOT                             63800 non-null  int64 
 7   EASE-MENT                       63800 non-null  object
 8   BUILDING CLASS AT PRESENT       63800 non-null  object
 9   ADDRESS                         63800 non-null  object
 10  APARTMENT NUMBER                63800 non-null  object
 11  ZIP CODE                        63800 non-null  int64 
 12  RESIDENTIAL UNITS               63800 non-null  int

In [35]:
n_bangunan2 = 63800

In [36]:
#cari proporsi
p_bangunan2 = n_bangunan2 / n_bangunanall
p_bangunan2

0.7546009367459905

In [37]:
nobs =  n_bangunanall #nobs adalah jumlah seluruh bangunan
count = n_bangunan2 #jumlah bangunan yang dibangun pada tahun 1900 - 2000
value = p_bangunan1 #proporsi bangunan yang dibangun pada tahun lebih dari 2000
proportions_ztest(count, nobs, value, alternative='larger', prop_var=False)

(416.2371732909096, 0.0)

In [38]:
zstat, pvalue = proportions_ztest(count, nobs, value, alternative='larger', prop_var=False)

print("p-value:", pvalue)

# kita bandingkan dengan alpha
alpha = 0.05

if (pvalue < alpha):
  print("REJECT null hypothesis")
else:
  print("Fail to Reject null hypothesis")

p-value: 0.0
REJECT null hypothesis


Hipotesis ditolak

**OVERALL ANALYSIS**

Terdapat perbedaan harga pada bangunan yang dibangun pada tahun 1900 - 2000 dengan bangunan yang dibangun pada tahun setelah 2000, dapat dilihat dari rata - rata, nilai tengah, range harga dan kisaran harganya. Hipotesis bahwa lebih banyak bangunan yang dibangun pada tahun lebih dari 2000 daripada bangunan yang dibangun pada tahun 1900 - 2000 adalah tidak benar.
